In [6]:
import math
from scipy import stats

def cohens_d_independent(m1, s1, n1, m2, s2, n2):
    # 1. pooled SD
    s1_sq = s1**2
    s2_sq = s2**2
    pooled_var = ((n1-1)*s1_sq + (n2-1)*s2_sq) / (n1+n2-2)
    s_pooled = math.sqrt(pooled_var)
    
    # 2. Cohen's d
    diff = m1 - m2
    d = diff / s_pooled
    
    return d

# Example
m1, s1, n1 = 80, 6, 15    # Group 1: mean, std, n
m2, s2, n2 = 75, 6, 15    # Group 2: mean, std, n

d = cohens_d_independent(m1, s1, n1, m2, s2, n2)

In [7]:
from scipy.stats import norm

alpha = 0.05   # type I error rate (1-alpha confidence level)
beta = 0.2     # type II error rate (1 - power)
norm.ppf(1 - alpha / 2)  # two-tailed z critical value for 95% confidence interval
norm.ppf(1 - beta)  # z critical value for 80% power

# Sample size calculation for one-sample z-test for mean
def calculate_sample_size(d, alpha, beta):
    z_alpha = norm.ppf(1 - alpha / 2)  # two-tailed
    z_beta = norm.ppf(1 - beta)
    n = 2 * (z_alpha + z_beta) ** 2 / d ** 2
    return int(n) + 1  # round up to next whole number

calculate_sample_size(d, alpha, beta)  # Example usage

23

# Experiment design

Generates a complete experimental design for assigning videos and beliefs to subjects. It creates all possible permutations of video orders (`all_orders`) and assigns random beliefs (`all_beliefs`) to each video for every subject. The design matrix (`design`) combines video order, belief, order index, and subject ID into a structured format. Finally, it prints the design in a tabular format, showing the video, belief, order, and subject for each assignment.

In [8]:
import numpy as np
from itertools import permutations

video_values = [
    '4171487-uhd_3840_2160_30fps.mp4', 
    '5768645-uhd_3840_2160_25fps.mp4', 
    '11946387_3840_2160_30fps.mp4', 
    '11946387_3840_2160_30fps.mp4'
]

belief_values = ['AI Generated', 'Human Generated']


videos = np.arange(1, len(video_values) + 1)
belief = np.arange(len(belief_values))
all_orders = np.array(list(permutations(videos)))
n_subjects = all_orders.shape[0]

# For each subject, assign a unique video order and random beliefs
all_beliefs = np.random.choice(belief, size=(n_subjects, videos.size))
orders = np.tile(np.arange(1, videos.size + 1), n_subjects)
subjects = np.repeat(np.arange(1, n_subjects + 1), videos.size)

design = np.column_stack((
    all_orders.flatten(),
    all_beliefs.flatten(),
    orders,
    subjects
))

print("Video\tBelief\tOrder\tSubject")
for i, row in enumerate(design):
    print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}")
    if (i + 1) % videos.size == 0:
        print()

Video	Belief	Order	Subject
1	0	1	1
2	0	2	1
3	1	3	1
4	1	4	1

1	0	1	2
2	0	2	2
4	1	3	2
3	0	4	2

1	1	1	3
3	1	2	3
2	1	3	3
4	0	4	3

1	1	1	4
3	1	2	4
4	1	3	4
2	0	4	4

1	1	1	5
4	1	2	5
2	1	3	5
3	0	4	5

1	1	1	6
4	0	2	6
3	1	3	6
2	1	4	6

2	1	1	7
1	1	2	7
3	0	3	7
4	1	4	7

2	1	1	8
1	1	2	8
4	1	3	8
3	1	4	8

2	1	1	9
3	1	2	9
1	1	3	9
4	1	4	9

2	0	1	10
3	0	2	10
4	1	3	10
1	1	4	10

2	0	1	11
4	1	2	11
1	0	3	11
3	0	4	11

2	1	1	12
4	0	2	12
3	0	3	12
1	1	4	12

3	1	1	13
1	1	2	13
2	1	3	13
4	0	4	13

3	0	1	14
1	1	2	14
4	0	3	14
2	1	4	14

3	0	1	15
2	1	2	15
1	0	3	15
4	0	4	15

3	1	1	16
2	1	2	16
4	1	3	16
1	1	4	16

3	1	1	17
4	0	2	17
1	1	3	17
2	0	4	17

3	1	1	18
4	1	2	18
2	0	3	18
1	1	4	18

4	0	1	19
1	0	2	19
2	0	3	19
3	0	4	19

4	1	1	20
1	1	2	20
3	1	3	20
2	0	4	20

4	0	1	21
2	1	2	21
1	0	3	21
3	0	4	21

4	1	1	22
2	1	2	22
3	0	3	22
1	0	4	22

4	0	1	23
3	0	2	23
1	1	3	23
2	1	4	23

4	1	1	24
3	1	2	24
2	1	3	24
1	1	4	24



In [9]:
design.shape, design.sum(axis=0)  # Sum of each column

((96, 4), array([ 240,   58,  240, 1200]))

Outputs the design in a CSV file that can be imported in PsychoPy

In [10]:
import csv
import numpy as np
out_path = "./subjects_design.csv"
header = ["video1","label1","video2","label2","video3","label3","video4","label4"]
subjects = np.unique(design[:, 3])
with open(out_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(header)
    for s in subjects:
        rows = design[design[:, 3] == s]
        rows = rows[rows[:, 2].argsort()]  # sort by Order
        row = []
        for r in rows:
            row.append(video_values[r[0] - 1])  # video
            row.append(belief_values[r[1]])     # label/belief
        w.writerow(row)
print(f"Saved CSV to: {out_path}")

Saved CSV to: ./subjects_design.csv
